---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [178]:
def get_energy(_debug=False):
    if _debug: print('\nget_energy:')

    # Load data
    import pandas as pd
    # 1. skip header and footer; 2. NaN data is '...'
    df1 = pd.read_excel("Energy Indicators.xls", skiprows=17, skipfooter=38, na_values="...")
    if _debug: print('Type: {}'.format(type(df1)))  # dataframe

    # Correct column names
    df1 = df1.drop(df1.columns[[0, 1]], axis=1)  # remove useless columns
    df1.rename(columns={df1.columns[0]: 'Country'}, inplace=True)
    df1.rename(columns={df1.columns[1]: 'Energy Supply'}, inplace=True)
    df1.rename(columns={df1.columns[2]: 'Energy Supply per Capita'}, inplace=True)
    df1.rename(columns={df1.columns[3]: '% Renewable'}, inplace=True)

    if _debug: print('df1 Columns: {}'.format(df1.columns))

    # Clean data
    import re
    rename_country1 = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"}

    df1['Energy Supply'] *= 1000000

    # 理论上DataFrame不应该做循环处理, 看看怎么改进
    country = pd.Series(len(df1))  # to set value
    for i in range(len(df1)):
        row = df1.loc[i, :]

        # Rename country name:
        # Step 1: remove number. e.g. Greenland7 -> Greenland
        country[i] = re.sub("\d", "", row['Country'])

        # Step 2: remove (), break the row and only keep index=0
        country[i] = country[i].split(' (')[0]

        # Step 3: replace name by rename_country
        if country[i] in rename_country1.keys():
            country[i] = rename_country1.get(country[i])
            if _debug: print('{} -> {}'.format(row['Country'], country[i]))
    df1['Country'] = country

    df1.set_index('Country', inplace=True)
    if _debug:
        print('df1 Columns: {}'.format(df1.columns))
        print(df1.loc['China'])
        print(df1.loc['South Korea'])
        # print(list(df1.index))
    return df1


def get_gdp(_debug=False):
    if _debug: print('\nget_gdp:')
    # Load data
    import pandas as pd
    df2 = pd.read_csv('world_bank.csv', skiprows=4, encoding="utf8")
    if _debug: print('df2 Columns: {}'.format(df2.columns))

    # Only needs country name and last 10 years data
    df2 = df2[['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
               '2015']]  # Here is [[]]
    df2.rename(columns={'Country Name': 'Country'}, inplace=True)
    if _debug: print('df2 Columns: {}'.format(df2.columns))

    # Clean data
    rename_country2 = {"Korea, Rep.": "South Korea",
                       "Iran, Islamic Rep.": "Iran",
                       "Hong Kong SAR, China": "Hong Kong"}

    country = pd.Series(len(df2))  # to set value
    for i in range(len(df2)):
        row = df2.iloc[i, :]
        # replace name by rename_country
        country[i] = row['Country']
        if country[i] in rename_country2.keys():
            country[i] = rename_country2.get(country[i])
            if _debug: print('{} -> {}'.format(row['Country'], country[i]))
    df2['Country'] = country

    df2.set_index('Country', inplace=True)
    if _debug:
        print('df2 Columns: {}'.format(df2.columns))
        print(df2.loc['China'])
        print(df2.loc['South Korea'])
        # print(list(df2.index))
    return df2


def get_rank(_debug=False):
    if _debug: print('\nget_rank:')

    # Load data
    import pandas as pd
    df3 = pd.read_excel("scimagojr-3.xlsx")

    if _debug: print('df3 Columns: {}'.format(df3.columns))

    df3.set_index('Country', inplace=True)
    if _debug:
        print('df3 Columns: {}'.format(df3.columns))
        # print(list(df3.index))
    return df3


def answer_one(_debug=False):
    energy = get_energy(_debug)
    GDP = get_gdp(_debug)
    ScimEn = get_rank(_debug)
    ScimEn = ScimEn[ScimEn['Rank'] < 16]  # top 15

    # left join by the index
    import pandas as pd
    df = pd.merge(ScimEn, energy, how='inner', left_index=True, right_index=True)
    df = pd.merge(df, GDP, how='inner', left_index=True, right_index=True)

    if _debug:
        print('df Length: {}'.format(len(df)))
        print('df Columns: {}'.format(df.columns))
        print('{}'.format(df.columns == ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                                         'Citations per document', 'H index', 'Energy Supply',
                                         'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
                                         '2009', '2010', '2011', '2012', '2013', '2014', '2015']))
        # print(df)
    return df

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [179]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [180]:
def answer_two():
    energy = get_energy()
    GDP = get_gdp()
    ScimEn = get_rank()
    print('Length: energy={}, GDP={}, ScimEn={}'.format(len(energy), len(GDP), len(ScimEn)))
    # Length: energy = 227, GDP = 264, ScimEn = 191

    # outer join by index
    import pandas as pd
    df_inner = pd.merge(energy, GDP, how='inner', left_index=True, right_index=True)
    df_inner = pd.merge(df_inner, ScimEn, how='inner', left_index=True, right_index=True)

    df_outer = pd.merge(energy, GDP, how='outer', left_index=True, right_index=True)
    df_outer = pd.merge(df_outer, ScimEn, how='outer', left_index=True, right_index=True)

    # print(list(df_outer.index))
    return df_outer.shape[0] - df_inner.shape[0]

### Question 3 (6.6%)
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [181]:
def answer_three():
    Top15 = answer_one()
    years = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    return Top15[years].mean(axis=1).sort_values(ascending=False)

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [182]:
def answer_four():
    Top15_avg = answer_three()
    Top6_country = Top15_avg.index[5]  # index start at 0
    print(Top6_country)  # United Kingdom

    Top15 = answer_one()
    Top6 = Top15.loc[Top6_country]
    diff = Top6['2015'] - Top6['2006']
    print(type(diff))  # <type 'numpy.float64'>
    return diff

### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [183]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean(axis=0)

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [184]:
def answer_six():
    Top15 = answer_one()
    max_country = Top15['% Renewable'].argmax()
    country = Top15.loc[max_country]
    return max_country, country['% Renewable']

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [185]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_country = Top15['Ratio'].argmax()
    country = Top15.loc[max_country]
    return max_country, country['Ratio']

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [186]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['Population'].sort_values(ascending=False).index[2]

### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [187]:
def answer_nine():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable documents per Capita'] = Top15['Citable documents'] / Top15['Population']
    return Top15['Citable documents per Capita'].corr(Top15['Energy Supply per Capita'])

In [188]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [189]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [190]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()  # 17.02028
    Top15['HighRenew'] = [1 if x >= median else 0 for x in Top15['% Renewable']]
    return Top15['HighRenew']

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [191]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'}
    Top15['Continent'] = [ContinentDict[x] for x in Top15.index]

    # init
    continents = sorted(set(ContinentDict.values()))
    columns = ['size', 'sum', 'mean', 'std']
    import pandas as pd
    df = pd.DataFrame(
        [pd.Series() * len(continents)], index=continents, columns=columns)

    # population
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    for c in continents:
        population_of_country_in_continent = Top15[Top15['Continent'] == c]['Population']
        df.set_value(c, ['size'], population_of_country_in_continent.count())
        df.set_value(c, ['sum'], population_of_country_in_continent.sum())
        df.set_value(c, ['mean'], population_of_country_in_continent.mean())
        df.set_value(c, ['std'], population_of_country_in_continent.std())
    return df

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [192]:
def answer_twelve():
    Top15 = answer_one()

    # % Renewable bin
    import pandas as pd
    Top15['bins for % Renewable'] = pd.cut(Top15['% Renewable'], 5)

    # continent
    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'}
    Top15['Continent'] = [ContinentDict[x] for x in Top15.index]

    Top15 = Top15.reset_index()
    import numpy as np
    df = Top15.groupby(['Continent', 'bins for % Renewable']).agg({'Country': np.count_nonzero})
    # print(df)
    #                                     Country
    # Continent     bins for % Renewable
    # Asia          (2.212, 15.753]             4
    #               (15.753, 29.227]            1
    # Australia     (2.212, 15.753]             1
    # Europe        (2.212, 15.753]             1
    #               (15.753, 29.227]            3
    #               (29.227, 42.701]            2
    # North America (2.212, 15.753]             1
    #               (56.174, 69.648]            1
    # South America (56.174, 69.648]            1
    return df.T.iloc[0]  # convert dataframe vector to series

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [193]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['Population'].apply(lambda x: "{:,}".format(x))

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [194]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [195]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!